##### 学习概要
###### 什么是LRU
    - Least Recent Used
    - 缓存淘汰算法的一种常用策略
    - 最近使用的数据是最有用的，很久都没有使用的数据是无用的，内存满了以后，优先删除无用的数据
    - 也可以说是一种特殊的数据结构，设capacity为最大容量，put存，get取，没有返回-1
    - put和get的时间复杂度均为O(1)
###### 算法思想
    - 哈希双向链表，双向链表和哈希表的结合，这样才能保证O(1)
    - get(val)
        - key不存在，返回-1
        - key存在，返回val，将key对应的Node提到开头
    - put(val)
        - 根据val，组合一个新Node
        - key存在
            - 删除（hash和nodeList均删除）
            - Node插入nodeList到开头
        - key不存在
            - 若cache已满
                - 删除末尾（hash和nodeList均删除）
            - Node插入nodeList到开头，hash中新建一个映射
###### 算法实现

##### LRU结构示意图

In [7]:
%%html
<img src="./images/LRU.png", width=400, heigth=600>

##### 算法实现

In [1]:
#双向链表的节点
class Node():
    def __init__(self,k,v):
        self.k = k
        self.v = v
        self.next = None
        self.prev = None

In [5]:
#双向链表
class DoubleList():
    def __init__(self):
        #作为表头的临时存储变量
        self._head = None
        
    def add_first(self,key,val):
        node = Node(key,val)
        if self.is_empty():
            #空链表
            self._head = node
        else:
            #将node的next指向_head，作为头结点
            node.next = self._head
            #将_head的prev指向node
            self._head.prev = node
            #此时node为首节点，将_head指向node
            self._head = node
            
    def remove(self,node):
        #为空，直接返回
        #不为空
        #判断_head是否为item
        #是，直接删除，并将_head的next作为_head
        #否，遍历_head.next，node_i相等，将node_i-1的next更新为node_i+1，node_i+1的prev更新为node_i-1
        if self.is_empty():
            return
        curr = self._head
        if curr.k == node.k:
            #链表中只有这一个节点
            if not curr.next:
                self._head = None
            else:
                self._head = self._head.next
                self._head.prev = None
        else:
            while curr:
                if curr.k == node.k:
                    curr.prev.next = curr.next
                    curr.next.prev = curr.prev
                    return
                curr = curr.next
            
    def removeLast(self):
        if self.is_empty():
            return None
        #只有一个节点
        del_key = None
        if not self._head.next:
            del_key = self._head.key
            self._head = None
        else:
            curr = self._head
            while curr:
                #最后一个节点
                if not curr.next:
                    del_key = curr.key
                    curr.prev.next = None
                curr = curr.next
        return del_key
        
    def size(self):
        if self.is_empty():
            return 0
        count = 1
        while self._head.next:
            count += 1
        return count
    
    def is_empty(self):
        if not self._head:return True
        else:return False

In [6]:
#LRU
class LRUCache():
    def __init__(self,capacity):
        self.capacity = capacity
        self.map = dict()
        self.cache = DoubleList()
    def get(self,key):
        if key not in self.map:
            return -1
        val = self.map[key].val
        #利用put方法将key对应的node提前
        self.put(key,val)
        return val
    def put(self,key,val):
        #根据key，val创建一个节点node
        #如果key在map中
        #DoubleList中删除key对应的节点，map更新key的值为node
        #如果key不在map中
        #DoubleList末尾新增node，map新增node
        node = Node(key,val)
        if key in self.map:
            self.cache.remove(node)
            self.cache.add_first(node)
            self.map[key] = node
        else:
            if self.cache.size() == self.capacity:
                del_key = self.cache.removeLast()
                self.map.pop(del_key)
            self.cache.add_first(node)
            self.map[key] = node